In [2]:
import numpy as np
import gurobipy as gp

In [17]:
def create_model(A, sense, b, obj, opt=gp.GRB.MAXIMIZE, ub=None, lb=None):
    # creating model
    model = gp.Model()

    # creating variable an setting the constraints
    if ub is None:
        modx = model.addMVar(A.shape[1])
    else:
        modx = model.addMVar(A.shape[1], ub=ub)
    mod_con = model.addMConstrs(A, modx, sense, b)

    # setting the objective function
    model.setMObjective(None, obj, 0, sense=opt)
 
    # restricting gurobi logs
    model.Params.OutputFlag = 0

    # optimizing the function
    model.optimize()
    
    return model

## Train wagon problem

In [18]:
# creating the objective array
obj = np.array([2000, 2500, 5000, 3500]*3)

# creating the constraint matrix
A = np.zeros((10, 12))

# creating the total tons constrains
A[0, :4] = 1
A[1, 4:8] = 1
A[2, 8:] = 1

# volume constraints for each wagon
A[3, :4] = [400, 300, 200, 500]
A[4, 4:8] = [400, 300, 200, 500]
A[5, 8:] =  [400, 300, 200, 500]

# total tons in each car
A[6, 0] = 1
A[6, 4] = 1
A[6, 8] = 1

A[7, 1] = 1
A[7, 5] = 1
A[7, 9] = 1

A[8, 2] = 1
A[8, 6] = 1
A[8, 10] = 1

A[9, 3] = 1
A[9, 7] = 1
A[9, 11] = 1

In [19]:
# constraint limits
b = np.array([10, 8, 12, 5000, 4000, 8000, 18, 10, 5, 20])

# direction of constraints
sense = np.array(['<'] * 10)

In [20]:
train_model = gp.Model()

train_modx = train_model.addMVar(A.shape[1])
train_mod_con = train_model.addMConstrs(A, train_modx, sense, b)

train_model.setMObjective(None, obj, 0, sense=gp.GRB.MAXIMIZE)

train_model.Params.OutputFlag = 0

In [21]:
train_model.optimize()

In [22]:
train_model.x

[0.0, 5.0, 5.0, 0.0, 0.0, 0.0, 0.0, 8.0, 0.0, 0.0, 0.0, 12.0]

In [23]:
train_model.objVal

107500.0

## Oil problem

In [10]:
obj = np.array([20, 15])

A = np.zeros((5, 2))

A[0, :] = [0.3, 0.4]
A[1, :] = [0.4, 0.2]
A[2, :] = [0.2, 0.3]
A[3, 0] = 1
A[4, 1] = 1

b = [2000, 1500, 5000, 9000, 6000]

sense = ['<', '<', '<', '<', '<']

In [11]:
oil_model = gp.Model()

oil_modx = oil_model.addMVar(A.shape[1])
oil_mod_con = oil_model.addMConstrs(A, oil_modx, sense, b)

oil_model.setMObjective(None, obj, 0, sense=gp.GRB.MAXIMIZE)

oil_model.Params.OutputFlag = 0

In [12]:
oil_model.optimize()

In [13]:
oil_model.objVal

92500.0

In [14]:
oil_model.x

[2000.0, 3500.0]

## Short term financing

In [30]:
A = np.zeros((6, 14))
b = np.zeros(6)

# first constraint
A[0, 0] = 1
A[0, 5] = 1
A[0, 8] = -1
b[0] = 150

# second constraint
A[1, 0] = -1.01
A[1, 1] = 1
A[1, 6] = 1
A[1, 8] = 1.003
A[1, 9] = -1
b[1] = 100

# third constraint
A[2, 1] = -1.01
A[2, 2] = 1
A[2, 7] = 1
A[2, 9] = 1.003
A[2, 10] = -1
b[2] = -200

# forth constraint
A[3, 2] = -1.01
A[3, 3] = 1
A[3, 5] = -1.02
A[3, 10] = 1.003
A[3, 11] = -1
b[3] = 200

# fift constraint
A[4, 3] = -1.01
A[4, 4] = 1
A[4, 6] = -1.02
A[4, 11] = 1.003
A[4, 12] = -1
b[4] = -50

# sixth constraint
A[5, 4] = -1.01
A[5, 7] = -1.02
A[5, 12] = 1.003
A[5, 13] = -1
b[5] = -300

# direciton of constraint
sense = ['=']*6

# upper bounds
ub = [100]*5 + [np.inf]*9

# objective function
c = np.array([0]*13 + [1])


In [33]:
short_fin_model = create_model(A, sense, b, c, opt=gp.GRB.MAXIMIZE, ub=None)

<ipython-input-33-583559b903de>:1: DeprecationWarning: Deprecated, use Model.addMConstr() instead
  short_fin_model = create_model(A, sense, b, c, opt=gp.GRB.MAXIMIZE, ub=None)


In [41]:
# bank loans
print('bank loans')
print(short_fin_model.x[:5])

# commercial paper
print('commercial paper')
print(short_fin_model.x[5:9])

# excess money
print('excess money')
print(short_fin_model.x[9:])

bank loans
[0.0, 50.98039215686275, 0.0, 0.0, 0.0]
commercial paper
[150.0, 49.01960784313725, 203.4343635759389, 0.0]
excess money
[0.0, 351.9441674975075, 0.0, 0.0, 92.49694915254233]


In [40]:
# final return
short_fin_model.objVal

92.49694915254233